In [1]:
from util import *
from PatchGan import *
from PIL import Image
from skimage import color

In [2]:
in_chan = 1 # input channels
real_chan = 2 # output channels, should be 3 if use rgb, be 2 if use lab
shape = 128 # shape of input image of our model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt_root = "./checkpoint/" # directory to load and save ckpt
classes = 365 # number of classes for our classfier

In [3]:
gen = UNet_Classfier(in_chan, real_chan, classes).to(device)
ckpt_path = ckpt_root + "UNet_Gan_v4_4.pth"

loaded_state = torch.load(ckpt_path, map_location = device)
gen.load_state_dict(loaded_state["gen"])

<All keys matched successfully>

In [134]:
# Please provide images that its shape is larger than (128,128)
# will return a PIL object
def colorized_image(path):
    # Paramaters:
    # path: str, the path of the image
    orig_img = Image.open(path)

    if len(np.array(orig_img).shape) == 3:
        img = np.array(orig_img.resize((shape, shape), resample = 3))
        img = color.rgb2lab(img).transpose((2,0,1))
        
        original_l = color.rgb2lab(np.array(orig_img))[:,:,0,None]
        
    else:
        img = np.array(orig_img.resize((shape, shape), resample = 3))[:, :, None]
        img = np.concatenate([img, img, img], axis = 2)
        img = color.rgb2lab(img).transpose((2,0,1))
        
        original = np.array(orig_img)[:,:,None]
        original = np.concatenate([original, original, original], axis = 2)
        original_l = color.rgb2lab(original)[:,:,0,None]
        
    l = img[0,:,:][None, None, :, :]    
    _, predicted_ab = gen(torch.tensor(l).float())
    predicted_ab = predicted_ab.detach().cpu().numpy()[0]
    lab = np.concatenate([l[0], predicted_ab], axis = 0).transpose((1,2,0))
    rgb = (color.lab2rgb(lab) * 255).astype("uint8")
    
    rescaled_img = Image.fromarray(rgb).resize(orig_img.size, 3)
    rescaled_ab = color.rgb2lab(np.array(rescaled_img))[:,:,1:]
    rescaled_rgb = color.lab2rgb(np.concatenate([original_l, rescaled_ab], axis = 2))
    
    return Image.fromarray((rescaled_rgb*255).astype("uint8"))

In [171]:
img_path = './Evaluation/14_i.png'
colorized_image(img_path).save('./Evaluation/14_i_p.png')